In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import nltk
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
import os
import glob
from collections import defaultdict
from tqdm import tqdm
import json

#import warnings
#warnings.filterwarnings("ignore")

In [2]:
# make dataframe holding disagreements

DATA_ROOT = "/Users/rpryzant/metascience/eLife/data/"


df = pd.read_csv(DATA_ROOT + "kyle/tolabel.csv", sep="|")
df = df[["Manuscript no.", "Reviewer ID", "CleanedComments", "Rec", "Suitable", "ShouldBe", "HumanLabel"]]
df = df.set_index(["Manuscript no."])
scored_bert = pd.read_csv(DATA_ROOT + "kyle/eval_results_full_allelife.txt", 
                          sep="\t", names=["id", "score", "dummy", "text"])


list(scored_bert.sort_values(by="score", ascending=False).iloc[1:10,]["text"])
df["score"] = list(scored_bert.score)
df["Text"] = list(scored_bert.text)




e = pd.read_csv(DATA_ROOT + "raw/eLife_Paper_history_2019_03_15.csv")
e["Manuscript no."] = e["ms"]
e = e.set_index(["Manuscript no."])
e = e.dropna(subset=["full_decision"])

# to get finaldecision, take last non-NA decision of the ones listed here
# note that this excludes rejected by initial decision
e["FinalDecision"] = e.apply(lambda x: list(x[["full_decision", "rev1_decision", "rev2_decision", "rev3_decision", "rev4_decision"]].dropna())[-1], axis=1)
e["outcome"] = np.where(e["FinalDecision"] == "Accept Full Submission", 1, 0)

df_e = df.join(e)



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# get disagreements

df_e["review_outcome"] = "none"
df_e["zscore"] = (df_e.score - np.mean(df_e.score))/np.std(df_e.score)
df_e.loc[(df_e.zscore > 1) & (df_e.outcome == 1), "review_outcome" ] = "pos_pos"
df_e.loc[(df_e.zscore > 1) & (df_e.outcome == 0), "review_outcome" ] = "pos_neg"
df_e.loc[(df_e.zscore < -1) & (df_e.outcome == 0), "review_outcome" ] = "neg_neg"
df_e.loc[(df_e.zscore < -1) & (df_e.outcome == 1), "review_outcome" ] = "neg_pos"

# papers with disagreement
disagreement = df_e.loc[(df_e.review_outcome == "pos_neg") | (df_e.review_outcome == "neg_pos")]
disagreement_papers = df_e.loc[set(disagreement.index)]

print(len(disagreement))
print(len(disagreement_papers))



2684
7021


In [5]:
disagreement_papers.loc[random.choice(disagreement_papers.index)]

,Reviewer ID,CleanedComments,Rec,Suitable,ShouldBe,HumanLabel,score,Text,ms,type,...,rev3_decision_dt,rev4_qc_dt,rev4_decision,rev4_decision_dt,p.poa_dt,p.vor_dt,FinalDecision,outcome,review_outcome,zscore
Manuscript no.,,,,,,,,,,,,,,,,,,,,,
34827,3163.0,This is a very nice piece of work that tackles...,False,False,False,NaN,3.323757,This is a very nice piece of work that tackles...,34827.0,RA,...,NaN,NaN,NaN,NaN,NaN,NaN,Reject Full Submission,0.0,pos_neg,1.135194
34827,101650.0,The manuscript explores the impact of stochast...,False,False,False,NaN,1.575787,The manuscript explores the impact of stochast...,34827.0,RA,...,NaN,NaN,NaN,NaN,NaN,NaN,Reject Full Submission,0.0,none,-0.776493
34827,104851.0,Wright and Vetsigian present a work whose cent...,False,False,False,NaN,0.963397,Wright and Vetsigian present a work whose cent...,34827.0,RA,...,NaN,NaN,NaN,NaN,NaN,NaN,Reject Full Submission,0.0,neg_neg,-1.446239


In [23]:
# load initial and reviewer consultations


def get_df(path_glob, sep="|"):
    out = None
    for file in glob.glob(path_glob):
        tmp = pd.read_csv(file, error_bad_lines=False, sep=sep) 
        if out is None:
            out = tmp
        else:
            out = out.append(tmp)
    out = out.set_index(["Manuscript no."])
    return out

initial_consults = get_df(
    DATA_ROOT + "raw/eLife_Initial_Consultation_*_PipeDelimited.txt")

review_consults = get_df(
    DATA_ROOT + "raw/eLife_Reviewer_Consultation_*_PipeDelimited.txt")

reviewers = get_df(DATA_ROOT + "raw/eLife_Reviewers.csv", sep=",")
authors = get_df(DATA_ROOT + "raw/eLife_Authors.csv", sep=",")
authors_orcids = get_df(DATA_ROOT + "raw/eLife_Authors_ORCIDS.csv", sep=",")
gender_reviewers = get_df(DATA_ROOT + "kyle/gender_reviewers.csv", sep=",")



b'Skipping line 9644: expected 11 fields, saw 13\nSkipping line 21637: expected 11 fields, saw 12\nSkipping line 21638: expected 11 fields, saw 12\nSkipping line 58138: expected 11 fields, saw 12\nSkipping line 58139: expected 11 fields, saw 12\nSkipping line 58142: expected 11 fields, saw 12\nSkipping line 58143: expected 11 fields, saw 12\n'
b'Skipping line 9911: expected 12 fields, saw 14\nSkipping line 22241: expected 12 fields, saw 13\nSkipping line 22242: expected 12 fields, saw 13\nSkipping line 59862: expected 12 fields, saw 13\nSkipping line 59863: expected 12 fields, saw 13\nSkipping line 59866: expected 12 fields, saw 13\nSkipping line 59867: expected 12 fields, saw 13\n'


In [33]:
gender_reviewers.set_index(["Reviewer ID"])
gender_reviewers.loc[1141].iloc[0].gender





'male'

In [21]:
# get into json with key is paper id and value is
# - paper id, outcome, author info
# - reviewer info
# - official reviews, initial consults, review consults
# author info is id, name, gender, institution, city

papers_dict = {}
for paper_id in set(disagreement_papers.index):
    # Skip if only 1 review
    if isinstance(disagreement_papers.loc[paper_id], pd.core.series.Series):
        continue

    papers_dict[paper_id] = {}

    def get_reviews():
        out = []
        
        for _, review in disagreement_papers.loc[paper_id].iterrows():
            tmp = {}
            tmp["text"] = review["CleanedComments"]
            tmp["score"] = review["score"]
            tmp["outcome"] = review["review_outcome"]
            tmp["author"] = str(int(review["Reviewer ID"]))
            out.append(tmp)
        return out
    
    def get_consults(df):
        out = []

        if paper_id not in df.index:
            return out

        # why is this happening
        if isinstance(df.loc[paper_id], pd.core.series.Series):
            return 

        # already sorted by time :)
        for _, review in df.loc[paper_id].iterrows():
            tmp = {}
            tmp["text"] = review["Comment text"]
            tmp["date"] = review["Comment date"]
            tmp["author"] = str(int(review["Commenter ID"]))
            out.append(tmp)
        return out

    def is_accept():
        if isinstance(disagreement_papers.loc[paper_id]["FinalDecision"], str):
            s = disagreement_papers.loc[paper_id]["FinalDecision"]
        else:
            s = disagreement_papers.loc[paper_id]["FinalDecision"].iloc[0]
        return 'Accept' in s

    def get_reviewers(reviews, target_outcome):
        out = []

        for review in reviews:
            if review["outcome"] == target_outcome:
                out.append(review["author"])
        return out

    papers_dict[paper_id]["reviews"] = get_reviews()
    papers_dict[paper_id]["initial_consults"] = get_consults(initial_consults)
    papers_dict[paper_id]["review_consults"] = get_consults(review_consults)

    papers_dict[paper_id]['is_accept'] = is_accept()
    papers_dict[paper_id]['authors'] = [str(int(x)) for x in list(authors.loc[3]["Author ID"])]
    papers_dict[paper_id]['winning_reviewers'] = \
        get_reviewers(papers_dict[paper_id]["reviews"], "neg_neg") + \
        get_reviewers(papers_dict[paper_id]["reviews"], "pos_pos")
    papers_dict[paper_id]['losing_reviewers'] = \
        get_reviewers(papers_dict[paper_id]["reviews"], "pos_neg") + \
        get_reviewers(papers_dict[paper_id]["reviews"], "neg_pos")
    papers_dict[paper_id]['neutral_reviewers'] = get_reviewers(papers_dict[paper_id]["reviews"], "none")



In [18]:
# get author metadata together...this takes a few minutes

def add_authors(d, df, role):

    for pid, row in df.iterrows():
        # every csv has person id as 3rd column, and manuscript id is not index
        try:
            person_id = int(row.iloc[1])
        except ValueError:
            continue
        pid = str(pid)
        if "pids" not in d[person_id]:
            d[person_id]["pids"] = [pid]
        else:
            d[person_id]["pids"] += [pid]

        if "roles" not in d[person_id]:
            d[person_id]["roles"] = [role]
        else:
            d[person_id]["roles"] += [role]

        print()
        
        # regexes because author/reviewer csvs are different
        d[person_id]["name"] = row.filter(regex=(".*name")).iloc[0]
        d[person_id]["email"] = row.filter(regex=(".*email")).iloc[0]
        d[person_id]["institution"] = row.filter(regex=(".*[Ii]nstitution")).iloc[0]
        
        city_series = row.filter(regex=("[Cc]ity"))
        if not city_series.empty:
            d[person_id]["city"] = city_series.iloc[0]

        country_series = row.filter(regex=("[Cc]ity"))
        if not country_series.empty:
            d[person_id]["country"] = country_series.iloc[0]

        
people_dict = defaultdict(dict)  
add_authors(people_dict, reviewers, "reviewer")
add_authors(people_dict, authors, "author")
add_authors(people_dict, authors_orcids, "author_orchid")
        


In [19]:
print(len(people_dict))
print(len(papers_dict))

66974
2433


In [22]:
with open('/Users/rpryzant/metascience/eLife/data/processed/papers.json', 'w') as fp:
    json.dump(papers_dict, fp)

with open('/Users/rpryzant/metascience/eLife/data/processed/people.json', 'w') as fp:
    json.dump(people_dict, fp)
    